In [18]:
import xml.etree.ElementTree as ET
import numpy as np
import meshio as mio
import json
import os

import meshplot as mp

In [19]:
def gmsh_dim_tags(points, cells, tags):
    dim_tags = np.tile([3, 1], [points.shape[0], 1])
    for i, cell in enumerate(cells):
        for vi in cell:
            dim_tags[vi, 1] = tags[i]
    return dim_tags


def gmsh_cell_tags(tags):
    cell_tags = [[tags[0]]]
    for tag in tags[1:]:
        if tag == cell_tags[-1][-1]:
            cell_tags[-1].append(tag)
        else:
            cell_tags.append([tag])
    return cell_tags


def split_cells_by_tags(tags, cells):
    assert(len(tags) == len(cells))
    split_cells = [[cells[0]]]
    for i, tag in enumerate(tags[1:]):
        if tag == tags[i]:
            split_cells[-1].append(cells[i + 1])
        else:
            split_cells.append([cells[i + 1]])
    return split_cells


def save_msh(file, points, tets, tags):
    dim_tags = gmsh_dim_tags(points, tets, tags)
    cell_tags = gmsh_cell_tags(tags)

    tmp = [("tetra", np.array(cells)) for cells in split_cells_by_tags(tags, tets)]
    # print([("tetra", T)])
    # [("tetra", T)]


    mesh = mio.Mesh(
        points,
        tmp,
        point_data={"gmsh:dim_tags": dim_tags},
        cell_data={"gmsh:physical": cell_tags, "gmsh:geometrical": cell_tags}
    )

    mio.write(file, mesh, binary=True, file_format="gmsh")

In [20]:
def load_control(control, args):
    if control is None:
        return
        
    tsn = control.find("time_steps")
    ssn = control.find("step_size")
    an = control.find("analysis")

    
    if tsn is not None and ssn is not None and an is not None:
        if an.attrib["type"] == "dynamic":
            time_steps = int(tsn.text)
            step_size = float(ssn.text)

            args["time"] = {
                "tend": step_size * time_steps,
                "time_steps": time_steps
            }

In [21]:
def load_materials(febio):
    materials = {}

    material_parent = febio.find("Material")
    
    for material_node in material_parent.iter("material"):
        material = material_node.attrib["type"]
        mid = int(material_node.attrib["id"])+1
        
        E = float(material_node.find("E").text)
        nu = float(material_node.find("v").text)
                  
        if material_node.find("density") is None:
            rho = 1
        else:
            rho = float(material_node.find("density").text)

        mat = ""
        if material == "neo-Hookean":
            mat = "NeoHookean"
        elif material == "isotropic elastic":
            mat = "LinearElasticity"
        else:
            print("Unsupported material {}, reverting to isotropic elastic".format(material))
            mat = "LinearElasticity"
            

        materials[mid] = {"id": mid, "E": E, "nu": nu, "rho": rho, "type": mat}
        
    return materials

In [22]:
def load_nodes(geometry):
    vertices = []
    for nodes in geometry.iter("Nodes"):
        for child in nodes.iter("node"):
            pos_str = child.text
            vs = pos_str.split(",")
            assert(len(vs) == 3);
            
            vertices.append([float(vs[0]), float(vs[1]), float(vs[2])])


    V = np.array(vertices)
    
    return V

In [23]:
def load_elements(geometry, numV, materials):
    els = []
    nodes = []
    mids = []
    
    order = 1
    
    is_hex = False
    types = ""
    
    for elements in geometry.iter("Elements"):
        el_type = elements.attrib["type"]
        mid = int(elements.attrib["mat"])+1

        if el_type != "tet4" and el_type != "tet10" and el_type != "tet20" and el_type != "hex8":
            print("Unsupported elemet type {}".format(el_type))
            continue


        if len(types) == 0:
            if el_type.startswith("tet"):
                types = "tet"
            else:
                types = "hex"
        elif not el_type.startswith(types, 0):
            print("Unsupported elemet type {} since the mesh contains also {}".format(el_type, types))
            continue

        if el_type == "tet4":
            order = max(1, order)
        elif el_type == "tet10":
            order = max(2, order)
        elif el_type == "tet20":
            order = max(3, order)
        elif el_type == "hex8":
            order = max(1, order)
            is_hex = True
                

        for child in elements.iter("elem"):
                ids = child.text
                tt = ids.split(",");
                assert(len(tt) >= 4);
                
                node_size = 8 if is_hex else 4

                els.append([])
                
                for n in range(node_size):
                    els[-1].append(int(tt[n]) - 1)
                    assert(els[-1][n] < numV)

                nodes.append([])
                mids.append(mid)
                
                for n in range(len(tt)):
                    nodes[-1].append(int(tt[n]) - 1)

                if el_type == "tet10":
                    assert(len(nodes[-1]) == 10)
                    nodes[-1][8], nodes[-1][9] = nodes[-1][9], nodes[-1][8]
                elif el_type == "tet20":
                    assert(len(nodes[-1]) == 20)
                    nodes[-1][8], nodes[-1][9] = nodes[-1][9], nodes[-1][8]
                    nodes[-1][10], nodes[-1][11] = nodes[-1][11], nodes[-1][10]
                    nodes[-1][12], nodes[-1][15] = nodes[-1][15], nodes[-1][12]
                    nodes[-1][13], nodes[-1][14] = nodes[-1][14], nodes[-1][13]
                    nodes[-1][16], nodes[-1][19] = nodes[-1][19], nodes[-1][16]
                    nodes[-1][17], nodes[-1][19] = nodes[-1][19], nodes[-1][17]


    T = np.array(els)
                
    return T, np.array(mids), order

In [24]:
def load_node_sets(geometry):
    nodes_set = {}
    
    for child in geometry.iter("NodeSet"):
        name = child.attrib["name"]
                
        for nodeid in child.iter("node"):
            nid = int(nodeid.attrib["id"])-1
            nodes_set[nid] = name

            
    return nodes_set

In [25]:
def load_dirichlet(boundaries, nodes_set, dt):
    allbc = {}
    
    names = [nodes_set[k] for k in nodes_set]
    
    result = np.zeros((len(nodes_set), 4))
    result[:, 0] = np.array([k for k in node_set])
    
    for child in boundaries.iter("fix"):
        name = child.attrib["node_set"]
        
        if not name in names:
            print("Sideset {} not present, skipping".format(name))
            continue

        bc = child.attrib["bc"]
        bcs = bc.split(",")

        value = np.array([0.,0.,0.])

        if not "x" in bcs:
            value[0] = np.NAN
        if not "y" in bcs:
            value[1] = np.NAN
        if not "z" in bcs:
            value[2] = np.NAN

        for i,k in enumerate(node_set):
            if node_set[k] == name:
                result[i, 1:] = value
                                

    for child in boundaries.iter("prescribe"):
        name = child.attrib["node_set"]
        if not name in names:
            print("Sideset {} not present, skipping".format(name))
            continue

        bc = child.attrib["bc"]
        bcs = bc.split(",")
        val = float(child.find("scale").text) * dt
        value = np.array([val, val, val])
        
        if not "x" in bcs:
            value[0] = np.NAN
        if not "y" in bcs:
            value[1] = np.NAN
        if not "z" in bcs:
            value[2] = np.NAN

        for i,k in enumerate(node_set):
            if node_set[k] == name:
                result[i, 1:] = value


#             for (const tinyxml2::XMLElement *child = boundaries->FirstChildElement("vector_bc"); child != NULL; child = child->NextSiblingElement("vector_bc"))
#             {
#                 const std::string name = std::string(child->Attribute("node_set"));
#                 if (names.find(name) == names.end())
#                 {
#                     logger().error("Sideset {} not present, skipping", name);
#                     continue;
#                 }
#                 const int id = names.at(name);
#                 const std::string centers = resolve_path(std::string(child->Attribute("centers")), root_file);
#                 const std::string values = resolve_path(std::string(child->Attribute("values")), root_file);
#                 const std::string rbf = "thin_plate"; //TODO
#                 const double eps = 1e-3;              //TODO
#                 //TODO add is x,y,z

#                 Eigen::MatrixXd centers_mat, values_mat;
#                 read_matrix(centers, centers_mat);
#                 read_matrix(values, values_mat);

#                 RBFInterpolation interp(values_mat, centers_mat, rbf, eps);
#                 logger().trace("adding vector Dirichlet id={} centers={} values={} rbf={} eps={}", id, centers, values, rbf, eps);

#                 gproblem.add_dirichlet_boundary(
#                     id, [interp](double x, double y, double z, double t) {
#                         Eigen::Matrix<double, 3, 1> v;
#                         v[0] = x;
#                         v[1] = y;
#                         v[2] = z;
#                         return interp.interpolate(v);
#                     },
#                     true, true, true, get_interpolation(gproblem.is_time_dependent()));
#             }

#             const bool is_time_dept = gproblem.is_time_dependent();
#             for (const tinyxml2::XMLElement *child = boundaries->FirstChildElement("scaling"); child != NULL; child = child->NextSiblingElement("scaling"))
#             {
#                 const std::string centres = std::string(child->Attribute("center"));
#                 const std::string factors = std::string(child->Attribute("factor"));
#                 const std::string name = std::string(child->Attribute("node_set"));
#                 if (names.find(name) == names.end())
#                 {
#                     logger().error("Sideset {} not present, skipping", name);
#                     continue;
#                 }
#                 const int id = names.at(name);

#                 const auto centrec = StringUtils::split(centres, ",");
#                 if (centrec.size() != 3)
#                 {
#                     logger().error("Skipping scaling, center is not 3d");
#                     continue;
#                 }
#                 const Eigen::Vector3d center(
#                     atof(centrec[0].c_str()),
#                     atof(centrec[1].c_str()),
#                     atof(centrec[2].c_str()));

#                 const double scaling = atof(factors.c_str());
#                 logger().trace("adding scaling Dirichlet id={} center=({}) scaling={}", id, center.transpose(), scaling);
#                 gproblem.add_dirichlet_boundary(
#                     id, [center, scaling, is_time_dept](double x, double y, double z, double t) {
#                         Eigen::Matrix<double, 3, 1> v;
#                         Eigen::Matrix<double, 3, 1> target;
#                         v[0] = x;
#                         v[1] = y;
#                         v[2] = z;
#                         target = v;

#                         const double s = is_time_dept ? (scaling * t) : scaling;
#                         target -= center;
#                         target *= s;
#                         target += center;
#                         return (target - v).eval();
#                     },
#                     true, true, true);
#             }
#         }

    return result

In [26]:
def load_neumann(loads, names, dt):
    result = []
    
    if loads is None:
        return result
    
    for child in loads.iter("surface_load"):
        name = child.attrib["surface"]
        btype = child.attrib["type"]
        assert(name in names)
        
        if btype == "traction":
            traction = child.find("traction").text
            scalev = np.array([1., 1., 1.])
            
            for scale in child.iter("scale"):
                scales = scale.text
                scalev *= float(scales)


            bcs = traction.split(",")
            assert(len(bcs) == 3)

            force = np.array([float(v) for v in bcs])
            force *= scalev*dt
            
            
            print("adding Neumann id={} force=({})".format(names[name], force))

            result.append({
                "type": "neumann",
                "id": names[name],
                "value": force
                #get_interpolation(gproblem.is_time_dependent())
            })
        
        elif btype == "pressure":
            pressures = child.find("pressure").text
            
            # TODO added minus here
            pressure = -float(pressures) * dt

            print("adding Pressure id={} pressure={}".format(names[name], pressure))
                        
            result.append({
                "type": "pressure",
                "id": names[name],
                "value": pressure
                #get_interpolation(gproblem.is_time_dependent())
            })
        else:
            print("Unsupported surface load {}".format(btype))

    
    return result


In [27]:
def load_surface_selection(geometry):
    n_id = 1
    names = {}
    
    selections = []
    
    for child in geometry.iter("Surface"):
        name = child.attrib["name"]
        names[name] = n_id;
        

        # TODO  only tri3
        for nodeid in child.iter("tri3"):
            ids = nodeid.text
            tt = ids.split(",")
            assert(len(tt) == 3)
            
            selections.append([n_id] + [int(v) - 1 for v in tt])

        for nodeid in child.iter("quad4"):
            ids = nodeid.text
            tt = ids.split(",")
            assert(len(tt) == 4)

            tmp.append([int(v) - 1 for v in tt])

        n_id += 1
            
    return np.array(selections), names

In [29]:
out_folder = ""
dhat = 0.001
output = "sim.vtu"

In [30]:
output_json = {}
output_json["output"] = {
    "json": "sim.json",
    "paraview": {
            "file_name": output,
            "surface": True,
            "options": {
                "material": True,
                "body_ids": True
            },
            "vismesh_rel_area": 10000000
        }
}

In [45]:
# tree = ET.parse("../data/test.feb")
tree = ET.parse("../data/lin-neo.feb")

root = tree.getroot()

if root.attrib["version"] != "2.5":
    assert(False)
    
control = root.find("Control")
load_control(control, output_json)

dt = 1

if "time" in output_json:
    dt = output_json["time"]["time_steps"]

materials = load_materials(root)

    
geometry = root.find("Geometry");

V = load_nodes(geometry)
T, mids, order = load_elements(geometry, V.shape[0], {})

node_set = load_node_sets(geometry)


boundaries = root.find("Boundary");
dirichlet = load_dirichlet(boundaries, node_set, dt)

surfs, names = load_surface_selection(geometry)
loads = root.find("Loads")
neumann = load_neumann(loads, names, dt)


has_collisions = geometry.find("SurfacePair")




#####################################################
################## Output ###########################
#####################################################

if has_collisions:
    output_json["contact"] = {
        "enabled": True,
        "dhat": dhat
    }


output_json["materials"] = [materials[k] for k in materials] if len(materials) > 1 else list(materials.values())[0]
output_json["boundary_conditions"] = {}


output_json["geometry"] = {
    "mesh": os.path.join(out_folder,"mesh.msh"),
    "surface_selection": "" if surfs.size <= 0 else os.path.join(out_folder,"surfaces.txt")
}

if surfs.size > 0:
    np.savetxt(os.path.join(out_folder,"surfaces.txt"), surfs, fmt='%d')

save_msh(os.path.join(out_folder,"mesh.msh"), V, T, mids)

if dirichlet.size > 0:
    output_json["boundary_conditions"]["dirichlet_boundary"] = [os.path.join(out_folder,"dirichlet.txt")]
    
    np.savetxt(os.path.join(out_folder,"dirichlet.txt"), dirichlet)
    
output_json["boundary_conditions"]["rhs"] = [0,0,0]

with open(os.path.join(out_folder, "sim.json"), "w") as f:
    f.write(json.dumps(output_json, indent="  "))

adding Neumann id=1 force=([0. 0. 1.])


In [46]:
!cat surfaces.txt

1 1714 418 1482
1 2210 1472 211
1 766 1683 1483
1 1391 2163 1390
1 1758 1953 1481
1 1461 458 1620
1 1156 2163 1391
1 1156 1384 2163
1 1208 870 1377
1 5 870 1208
1 1274 942 1392
1 1392 942 1214
1 1071 1382 427
1 1405 2112 1470
1 1275 1405 1470
1 2223 1161 1385
1 975 1395 1399
1 1402 871 1587
1 1385 1161 1384
1 1391 2272 975
1 1382 1384 1628
1 1418 1715 1377
1 1492 1475 211
1 211 1475 1614
1 1377 1388 1106
1 1461 1620 1082
1 1046 1619 1461
1 1619 458 1461
1 1393 1387 1214
1 766 1676 1404
1 1470 1472 2274
1 1470 1611 1472
1 1481 2275 1758
1 1214 1387 2271
1 1385 1156 1266
1 1292 1488 1611
1 1470 1292 1611
1 1676 1483 1958
1 1485 1561 2112
1 1392 1214 1266
1 1402 1587 766
1 1402 766 1404
1 766 1483 1676
1 1476 1680 1389
1 920 1382 1071
1 1614 418 211
1 1470 2112 1292
1 1405 1216 1404
1 1404 1127 1405
1 2163 1406 1797
1 2 1071 2316
1 1161 1628 1384
1 2223 1385 1250
1 1371 1994 1250
1 1082 1620 1829
1 2274 1472 1465
1 1363 1161 1364
1 1680 1208 1106
1 1465 1082 2274
1 1399 1275 1401
1 1404 1

In [52]:
asd=surfs[:, 1:]

In [78]:
np.where(1106 == T[:, 1])

(array([350, 438]),)

In [80]:
V[1106,:], 

array([-0.67464407,  0.74126373,  3.        ])

In [ ]:
def convert(T, c):
    f = np.ndarray([T.shape[0]*4, 3], dtype=T.dtype)
    outc = np.ndarray([T.shape[0]*4], dtype=c.dtype)
    
    for i in range(T.shape[0]):
        f[i*4+0] = np.array([T[i][1], T[i][0], T[i][2]])
        f[i*4+1] = np.array([T[i][0], T[i][1], T[i][3]])
        f[i*4+2] = np.array([T[i][1], T[i][2], T[i][3]])
        f[i*4+3] = np.array([T[i][2], T[i][0], T[i][3]])
        
        outc[i*4:i*4+4] = c[i]
        
    return f, outc

In [ ]:
f, c = convert(T, mids)
mp.plot(V, f, c, shading={"point_size": 1, "wireframe": True})

In [ ]:
? mio.Mesh

In [ ]:
root.find("Globals")